In [1]:
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

# Load environment variables from the .env file
load_dotenv()

# Get the MySQL credentials from environment variables
user = os.getenv('MYSQL_USER')
password = os.getenv('MYSQL_PASSWORD')
host = os.getenv('MYSQL_HOST')
port = os.getenv('MYSQL_PORT')
dbname = os.getenv('MYSQL_DB')

# Create the database URL
db_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{dbname}"

# Create an engine instance
try:
    engine = create_engine(db_url)
    connection = engine.connect()
    print("Connected to the database successfully")
    connection.close()
except Exception as e:
    print(f"Error: {e}")


Connected to the database successfully


In [6]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("DESC candidates;"))
        columns = result.fetchall()

        print(f"Structure of 'candidates' table in '{dbname}':")
        print(f"{'Field':<20}{'Type':<20}{'Null':<10}{'Key':<10}{'Default':<10}{'Extra':<20}")
        print("-" * 90)
        for column in columns:
            print(f"{column[0]:<20}{column[1]:<20}{column[2]:<10}{column[3]:<10}{column[4] or 'NULL':<10}{column[5]:<20}")
except Exception as e:
    print(f"Error executing query: {e}")

Structure of 'candidates' table in 'ws_001':
Field               Type                Null      Key       Default   Extra               
------------------------------------------------------------------------------------------
first_name          varchar(50)         YES                 NULL                          
last_name           varchar(50)         YES                 NULL                          
email               varchar(100)        YES                 NULL                          
country             varchar(50)         YES                 NULL                          
application_date    date                YES                 NULL                          
yoe                 int                 YES                 NULL                          
seniority           varchar(50)         YES                 NULL                          
technology          varchar(50)         YES                 NULL                          
code_challenge_scoreint                 YES  